# 0) vérifier [sc] et [spark]

In [1]:
# le spark context détermine les ressources 
# disponibles pour l'application
assert sc==sc

In [2]:
# la variable spark permet de gérer les
# dataframes (dépend de pyspark.sql)
assert spark==spark

In [3]:
spark

# 1) loader un fichier en mémoire

In [9]:
path  = "hdfs://spark52:8020/cycliste_debug_2.csv"
# si on prend un répertoire, alors la rdd comprendra tous les fichiers
# du répertoire
path = "/Users/romain/Informatique/notebooks/spark_scala/velib/logs/cycliste_cyclistes.csv" 

In [10]:
rdd  = sc.textFile(path)
logs = rdd

In [11]:
logs.count()

3301

## 1.1) essayer de comprendre le fichier

In [7]:
logs.count() # nombre de lignes

1538884

In [8]:
logs.take(5) # extrait

[u'cycliste,sportif,age,sexe,nb_km,vitesse,attente',
 u'cycliste_azthv,0,34,femme,9.86503821812,15.4581576429,0.5',
 u'cycliste_azqs2,4,63,homme,4.08962700688,24.4077572311,0.5',
 u'cycliste_azyuo,0,55,femme,11.6630585491,7.87417725815,0.5',
 u'cycliste_azrd7,0,18,femme,11.229659737,22.4958582464,0.5']

In [13]:
header = logs.take(1)
header = header[0]
header

u'cycliste,sportif,age,sexe,nb_km,vitesse,attente'

In [14]:
sans_header = logs.filter(lambda ligne: ligne != header)
assert(sans_header.count() == logs.count() -1)

#### 1.1)  nombre de virgules par lignes

In [15]:
separator       = ","
lambda_function = lambda log: len(log.split(separator))
nb_virgules     = logs.map(lambda_function)

In [16]:
nb_possible  = nb_virgules.distinct()
nb_possible.collect()

[1, 4, 5, 6, 7, 8]

##### 1.2) trouver des logs pour chaque nb de virgules

In [204]:
create_pair = lambda log: (len(log.split(separator)), log)
pairedRdd   = logs.map(create_pair) 
pairedRdd.take(3)

[(4, u'cycliste,n_message,time,message'),
 (4, u'cycliste_aztu4,0000, 1496170615.88,station_maison = azuf3'),
 (1, u'cree')]

In [205]:
pairedRdd.groupByKey().take(3) # renvoie un itérateur (ouf !)

[(4, <pyspark.resultiterable.ResultIterable at 0x112fb3910>),
 (1, <pyspark.resultiterable.ResultIterable at 0x10c99a950>),
 (5, <pyspark.resultiterable.ResultIterable at 0x10c99a850>)]

In [206]:
pairedRdd.countByKey() 

defaultdict(int, {1: 2314, 4: 202465, 5: 42795})

##### 1.2.1) on affiche pour chaque nb de virgules quelques exemples

In [207]:
key_len__iterateur = pairedRdd.groupByKey().take(3)
for key_len, iterateur in key_len__iterateur:
    print "="*20
    print "nb de virgules = ", key_len
    print "exemples : "
    for i in range(3):
        print iterateur.data[i]
        

nb de virgules =  4
exemples : 
cycliste,n_message,time,message
cycliste_aztu4,0000, 1496170615.88,station_maison = azuf3
cycliste_aztu4,0002, 1496170615.88,station_maison nearest= aeqc6|aeohw|azum7|aeyd1|artgc|arojk|aztop|aruk0
nb de virgules =  1
exemples : 
cree
cree
cree
nb de virgules =  5
exemples : 
cycliste_azil1,0006, 1496170625.95,velo defaillant <0.3, rendu a la meme station :-( 
cycliste_azij6,0007, 1496170626.51,velo defaillant <0.3, rendu a la meme station :-( 
cycliste_aztob,0177, 1496170283.43,impossible de rendre sur ,azfj8 ( a la maison = False)


#### 1.2.2 supprimeons les "cree"

In [208]:
logs_sans_cree = logs.filter(lambda log: log !="cree")
logs_sans_cree.count()

245260

In [209]:
log_a_5_virgules = logs_sans_cree.filter(lambda log: len(log.split(separator))==5)
log_a_5_virgules.count()

42795

In [210]:
log_des_impossibles = log_a_5_virgules.filter(lambda log : "impossible" in log)
log_des_impossibles.count()

36847

#### 1.3 Elements distincts entre deux rdd => substract

In [211]:
cinq_virgules_non_impossible = log_a_5_virgules.subtract(log_des_impossibles)
cinq_virgules_non_impossible.count()

5948

In [212]:
cinq_virgules_non_impossible.take(4)

[u'cycliste_aztu4,0038, 1496170669.26,velo defaillant <0.1, rendu a la meme station :-( ',
 u'cycliste_azri3,0089, 1496170700.58,velo defaillant <0.1, rendu a la meme station :-( ',
 u'cycliste_azesh,0089, 1496170715.58,velo defaillant <0.1, rendu a la meme station :-( ',
 u'cycliste_azyus,0056, 1496170688.25,velo defaillant <0.1, rendu a la meme station :-( ']

In [213]:
logs_tristes = cinq_virgules_non_impossible.filter(lambda log: ":-(" in log)
logs_tristes.count()

5948

In [214]:
logs_tristes.countApproxDistinct()

5763L

### 2) cyclistes

In [215]:
cyclistes = logs.filter(lambda log: log.startswith("cycliste"))
cyclistes.count()

245260

In [216]:
sorted_cyclistes = cyclistes.sortBy(lambda x: x)
sorted_cyclistes.take(2)

[u'cycliste,n_message,time,message',
 u'cycliste_aze12,0000, 1496170618.45,station_maison = azupx']

# Sample

In [217]:
nb_de_ligne_voulu = 10
nb_ligne_existant = sorted_cyclistes.count()
pourcentage_voulu = nb_de_ligne_voulu/float(nb_ligne_existant)
avec_remise       = False

In [218]:
sorted_cyclistes.sample(avec_remise, pourcentage_voulu).collect()

[u'cycliste_azeti,0068, 1496170696.54,velo pris sur station aryln (elem de station maison)',
 u'cycliste_azeyj,0088, 1496170710.16,velo pris sur station azekm (elem de station travail)',
 u'cycliste_azips,0040, 1496170671.58,velo rendu sur la station azekm ( a la maison = False) (elem de station_travail)',
 u'cycliste_aziq5,0089, 1496170695.53,impossible de rendre sur ,azrj7 ( a la maison = False)',
 u'cycliste_aziqc,0031, 1496170650.21,False a pris le velo velo_azioj_1496170615.21 sur la station aeqc6',
 u'cycliste_azix9,0083, 1496170719.47,pas encore arrive',
 u'cycliste_azqg4,0061, 1496170701.15,True a pris le velo velo_aeif3_1496170612.85 sur la station artc6',
 u'cycliste_azrd9,0082, 1496170717.87,velo pris sur station aekm8 (elem de station travail)',
 u'cycliste_azuip,0043, 1496170676.78,velo rendu sur la station aztj5 ( a la maison = True) (elem de station_maison)',
 u'cycliste_azuob,0008, 1496170629.73,self.a_la_maison = False',
 u'cycliste_azymb,0087, 1496170695.59,impossible

# Extraire les logs de début et fin de trajet

In [219]:
filtre = lambda log: "velo rendu" in log or "velo pris" in log 
debut_fin_trajet = sorted_cyclistes.filter(filtre)
debut_fin_trajet.take(10)

[u'cycliste_aze12,0004, 1496170627.84,velo pris sur station aepf4 (elem de station maison)',
 u'cycliste_aze12,0008, 1496170629.68,velo rendu sur la station azs58 ( a la maison = False) (elem de station_travail)',
 u'cycliste_aze12,0011, 1496170633.47,velo pris sur station azk49 (elem de station travail)',
 u'cycliste_aze12,0015, 1496170634.73,velo rendu sur la station arqh8 ( a la maison = True) (elem de station_maison)',
 u'cycliste_aze12,0016, 1496170638.5,velo pris sur station arqh8 (elem de station maison)',
 u'cycliste_aze12,0019, 1496170639.79,velo rendu sur la station aeyu0 ( a la maison = False) (elem de station_travail)',
 u'cycliste_aze12,0020, 1496170644.37,velo pris sur station azs58 (elem de station travail)',
 u'cycliste_aze12,0024, 1496170644.37,velo rendu sur la station azs58 ( a la maison = False) (elem de station_travail)',
 u'cycliste_aze12,0025, 1496170646.53,velo pris sur station azk49 (elem de station travail)',
 u'cycliste_aze12,0028, 1496170647.95,velo rendu su

# Extraire les heures de logs

In [220]:
def get_cycliste_et_timestamp_du_log(log):
    data      = log.split(",")
    cycliste  = data[0]
    timestamp = float(data[2])
    return (cycliste, timestamp)


In [221]:
cycliste_timestamps = debut_fin_trajet.map(get_cycliste_et_timestamp_du_log)
cycliste_timestamps.take(23)

[(u'cycliste_aze12', 1496170627.84),
 (u'cycliste_aze12', 1496170629.68),
 (u'cycliste_aze12', 1496170633.47),
 (u'cycliste_aze12', 1496170634.73),
 (u'cycliste_aze12', 1496170638.5),
 (u'cycliste_aze12', 1496170639.79),
 (u'cycliste_aze12', 1496170644.37),
 (u'cycliste_aze12', 1496170644.37),
 (u'cycliste_aze12', 1496170646.53),
 (u'cycliste_aze12', 1496170647.95),
 (u'cycliste_aze12', 1496170651.74),
 (u'cycliste_aze12', 1496170653.28),
 (u'cycliste_aze12', 1496170657.22),
 (u'cycliste_aze12', 1496170657.22),
 (u'cycliste_aze12', 1496170660.07),
 (u'cycliste_aze12', 1496170661.29),
 (u'cycliste_aze12', 1496170665.3),
 (u'cycliste_aze12', 1496170666.58),
 (u'cycliste_aze12', 1496170670.43),
 (u'cycliste_aze12', 1496170671.67),
 (u'cycliste_aze12', 1496170675.56),
 (u'cycliste_aze12', 1496170676.85),
 (u'cycliste_aze12', 1496170680.89)]

# grouper les durées par cycliste

In [222]:
timestamps_par_cycliste = cycliste_timestamps.groupByKey()

# Calculer les délais entre deux timestamp

In [223]:
def diff_timestamp(liste):
    """
    Retourne une liste de delta !
    """
    timestamps = liste
    r          = []
    values     = []
    for indice, valeur in enumerate(sorted(timestamps)):
        values.append(float(valeur))
        if indice>0:
            delta = values[indice] - values[indice-1]
            r.append(delta)
    return r

In [224]:
delais_par_cyclistes = timestamps_par_cycliste.mapValues(diff_timestamp)
delais_par_cyclistes.take(5) # tableau de tuples comprenant une liste

[(u'cycliste_azr20',
  [1.7799999713897705,
   3.8499999046325684,
   1.190000057220459,
   3.809999942779541,
   1.25,
   4.620000123977661,
   0.0,
   2.169999837875366,
   1.380000114440918,
   3.819999933242798,
   1.5,
   3.990000009536743,
   0.0,
   2.8299999237060547,
   1.190000057220459,
   4.039999961853027,
   1.2599999904632568,
   3.870000123977661,
   1.2100000381469727,
   3.9099998474121094,
   1.2799999713897705,
   4.06000018119812,
   1.3599998950958252,
   4.079999923706055,
   0.0,
   2.5200002193450928,
   1.309999942779541,
   3.9800000190734863,
   0.0,
   2.6399998664855957,
   1.380000114440918,
   3.9600000381469727,
   1.309999942779541,
   3.9800000190734863,
   1.2400000095367432,
   4.059999942779541,
   1.2999999523162842,
   4.140000104904175,
   1.309999942779541,
   3.9800000190734863,
   1.309999942779541]),
 (u'cycliste_azusd', []),
 (u'cycliste_azrd8',
  [3.190000057220459,
   2.930000066757202,
   2.1500000953674316,
   3.0899999141693115,
   1.9

# Sommer les délais pour chaque cycliste

In [245]:
delais_concatenes = delais_par_cyclistes.reduceByKey(concatenation_de_liste)
infos             = delais_concatenes.take(3)
for id_cycliste, durees in infos:
    print "--------------"
    print id_cycliste
    print "np.min(durees) = %.2f"%np.min(durees)
    print "np.max(durees) = %.2f"%np.max(durees)
    print "np.mean(durees)= %.2f"%np.mean(durees)
    print "np.sum(durees) = %.2f"%np.sum(durees)

--------------
cycliste_azr20
np.min(durees) = 0.00
np.max(durees) = 4.62
np.mean(durees)= 2.36
np.sum(durees) = 96.87
--------------
cycliste_azrd8
np.min(durees) = 0.00
np.max(durees) = 4.00
np.mean(durees)= 2.48
np.sum(durees) = 99.23
--------------
cycliste_azr24
np.min(durees) = 1.72
np.max(durees) = 3.79
np.mean(durees)= 2.66
np.sum(durees) = 98.60


In [246]:
delais_concatenes.partitionBy(100)

MapPartitionsRDD[360] at mapPartitions at PythonRDD.scala:422

In [247]:
temps_de_parcours = delais_par_cyclistes.mapValues(lambda x:sum(x))

In [250]:
temps_de_parcours.take(5)

[(u'cycliste_azr20', 96.86999988555908),
 (u'cycliste_azusd', 0),
 (u'cycliste_azrd8', 99.23000001907349),
 (u'cycliste_azr24', 98.60000014305115),
 (u'cycliste_azed3', 95.08999991416931)]

# Application d'une fonction distribuable ...

In [251]:
# on fait une opération associative et commutative (afin qu'elle soit distruable)
def somme_de_valeurs(v1, v2):
    return v1 + v2
def concatenation_de_liste (liste1, liste2):  
    if type(liste1)==type([]):
        return sum(liste1)+sum(liste2)
    else :
        return "pbm avec  %s"%type(liste1)
def moyenne_de_liste( x, y): 
    if type(liste1)==type([]):
        liste_concatennees = x + y
        return np.mean( liste_concatennees )
    else :
        return "pbm avec  %s"%type(liste1)
    
def max_de_liste( x, y): 
    if type(liste1)==type([]):
        liste_concatennees = x + y
        return np.max( liste_concatennees )
    else :
        return "pbm avec  %s"%type(liste1)    

# ... d'un partitionnement vers le même partitionnement

In [252]:
liste_par_defaut   = []

In [257]:
moyenne_ou_rien = lambda default,liste   : [np.mean(liste)] if liste else default 

duree_moyenne_par_cycliste = delais_concatenes.aggregateByKey(  liste_par_defaut ,
                                                                moyenne_ou_rien  ,
                                                                moyenne_de_liste )

duree_moyenne_par_cycliste.take(3)

[(u'cycliste_azr20', [2.3626829240380265]),
 (u'cycliste_azrd8', [2.4807500004768372]),
 (u'cycliste_azr24', [2.6648648687311121])]

In [254]:
max_ou_rien = lambda default,liste   : [np.max(liste)] if liste else default 

duree_max_par_cycliste = delais_concatenes.aggregateByKey(  liste_par_defaut ,
                                                                max_ou_rien  ,
                                                                max_de_liste )

duree_max_par_cycliste.take(3)

[(u'cycliste_azr20', [4.6200001239776611]),
 (u'cycliste_azrd8', [4.0]),
 (u'cycliste_azr24', [3.7899999618530273])]